In [1]:
import selenium 
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

In [2]:
def cleaning_partidos(s):
    s.replace(" ' ",'')
    if ')' in s: s = s.split(')')[0]
    if 'Partido Popular'==s: s = 'PP'
    if "C's"== s: s = 'Cs'
    return s

def scrap_wiki(nombre_pueblo, provincia='Burgos'):
    try:
        URL = f"https://es.wikipedia.org/wiki/{nombre_pueblo.replace(' ','_')}"
        r = requests.get(URL)
        #print(r)
        #print(URL)
        soup = BeautifulSoup(r.content, 'lxml') 
        bodies = soup.findAll('tbody')
        a = False
        for e in bodies:
            if 'Alcalde' in e.text:
                r_tabla = e.findAll('tr')
                a=True
        if not a: 
            URL = f"https://es.wikipedia.org/wiki/{nombre_pueblo.replace(' ','_')}_({provincia})"
            r = requests.get(URL)
            soup = BeautifulSoup(r.content, 'lxml') 
            bodies = soup.findAll('tbody')
            for e in bodies:
                if 'Alcalde' in e.text:
                    r_tabla = e.findAll('tr')
                    a=True
        if not a: 
            URL = f"https://es.wikipedia.org/wiki/{nombre_pueblo.replace(' ','_')}_(España)"
            r = requests.get(URL)
            soup = BeautifulSoup(r.content, 'lxml') 
            bodies = soup.findAll('tbody')
            for e in bodies:
                if 'Alcalde' in e.text:
                    r_tabla = e.findAll('tr')
                    a=True

        for i in r_tabla:
            if 'Alcalde' in i.text:
                alcalde = list(i)[1].text.strip('\n')
                if '(' in alcalde:
                    lli = alcalde.split('(')
                    alcalde = lli[0].strip()
                    partido_pol = lli[1].strip(')')
          
        return cleaning_partidos(partido_pol)
    except:
        print(nombre_pueblo)
        return np.nan

In [3]:
# como la idea es crearnos un dataset con todos los nombres de pueblos a estudiar vamos a proceder a 
# crear una serie de funciones que nos haga todo el proceso,
def cambio_nombre(s):
    if ',' in s:
        lli = [i.strip() for i in s.split(',')]
        return lli[1]+' '+lli[0]
    else: return s
    
def lectura_csv(name, sepa=';'):
    path = f'../Data/castilla_leon/poblacion/{name}.csv'
    df = pd.read_csv(path,sep=sepa)
    provincia = df.Municipios[0][2:].strip()
    df = df[(df['Periodo']==2021)&(df['Sexo']=='Total')].reset_index(drop=True)[1:]
    df['CP'] = df.Municipios.apply(lambda x: x[:5].strip())
    df.Municipios = df.Municipios.apply(lambda x: x[5:].strip())
    df.drop(columns=['Sexo', 'Periodo'], inplace=True)
    df.Municipios = df.Municipios.apply(cambio_nombre)
    df.Total = df.Total.apply(lambda x: int(str(x).replace('.','')))
    df['Provincia'] = provincia
    return provincia, df

In [5]:
prov, df = lectura_csv('2906',sepa=';')

In [6]:
df['new_col'] = df.Municipios.apply(scrap_wiki, args=[prov])

Abezames
Arcos de la Polvorosa
Aspariegos
Bustillo del Oro
Calzadilla de Tera
Fresno de la Ribera
Fuentesecas
Gallegos del Pan
Malva
Matilla la Seca
Morales de Toro
Muelas de los Caballeros
Otero de Bodas
Palacios de Sanabria
Peleagonzalo
Pinilla de Toro
Pozoantiguo
Quiruelas de Vidriales
Salce
Toro
Valdefinjas
Venialbo
Vezdemarbán


In [22]:
# # Para palencia
# diccio_alc = {'Autillo de Campos': 'PP','Carrión de los Condes':'Cs',
# 'Dueñas':'PSOE', 'Lomas':'PP','Valderrábano':'PP' }

# for k,v in diccio_alc.items():
#     df.loc[df.Municipios == k, 'new_col'] = v
prov

'Soria'

In [7]:
# # Para Salamanca
# diccio_alc = {'Garcibuey': 'PP','Negrilla de Palencia':'Cs',
# 'San Cristóbal de la Cuesta':'Cs', 'Saucelle':'PP'}

# for k,v in diccio_alc.items():
#     df.loc[df.Municipios == k, 'new_col'] = v
# # Para Segovia
# diccio_alc = {'Grajera': 'PSOE'}

# for k,v in diccio_alc.items():
#     df.loc[df.Municipios == k, 'new_col'] = v
# # Para Soria
# diccio_alc = {'Blacos': 'Cs','San Esteban de Gormaz':'PSOE'}

# for k,v in diccio_alc.items():
#     df.loc[df.Municipios == k, 'new_col'] = v

# Para valladolid
# diccio_alc = {'Cistérniga': 'PSOE','Melgar de Abajo':'JPM','San Salvador':'PP','Valdestillas':'No Adscrito'}

# for k,v in diccio_alc.items():
#     df.loc[df.Municipios == k, 'new_col'] = v
# Para Zamora
diccio_alc = {'Abezames': 'PSOE','Arcos de la Polvorosa':'PP','Aspariegos':'PP','Bustillo del Oro':'PP','Bustillo del Oro':'PP'}

for k,v in diccio_alc.items():
    df.loc[df.Municipios == k, 'new_col'] = v

In [8]:
c_autonoma = 'castilla_leon'
#df.to_csv(f'../Data/{c_autonoma}/ParaBD/{prov}.csv')
df.to_csv(f'../Data/{c_autonoma}/ParaBD/{prov}.csv', mode='w', header=False)

In [9]:
df.head()

,Municipios,Total,CP,Provincia,new_col
1,Abezames,580,49002,Zamora,PSOE
2,Alcañices,1042,49003,Zamora,PP
3,Alcubilla de Nogales,1170,49004,Zamora,PSOE
4,Alfaraz de Sayago,1370,49005,Zamora,Ahora Decide
5,Algodre,1350,49006,Zamora,PP


In [10]:
df.to_csv(f'../Data/{c_autonoma}/ParaBD/Total.csv', mode='a', header=False)